# Graphanalyse Erfinderbeziehungen
Zunächst muss die folgende Zelle zum Laden der Module ausgeführt werden. Danach müssen die weiteren Zellen nach und nach ausgeführt werden.

In [1]:
import os
import pandas as pd
from collections import Counter, OrderedDict
import networkx as nx
import nx_altair as nxa
import altair as alt
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt

# Daten einlesen
Der fogende Code liest den Export der PatentsView-Datenbank aus und speichert diese in einem Pandas-Dataframe.

In [2]:
# load data
filepath = os.path.join(os.getcwd(), 'Datensatz_Erfinder_raw.csv')
data = pd.read_csv(filepath , sep = ";", encoding="mbcs")

# Daten filtern
Um den späteren Gesamtgraphen übersichtlicher zu gestalten, werden nur diejenigen Patentanmelder in den Graphen aufgenommen, die mindestens 10 Patente besitzen.

In [3]:
# count companies patents
patent_counter = Counter()
for row in data.itertuples():    
    row_df = pd.DataFrame(row).transpose().drop(0, axis=1)
    row_df.columns = data.columns            
    patent_counter[row_df["Anmelder/Inhaber"][0]] += 1
    
patent_counter = Counter({k: c for k, c in patent_counter.items() if c >= 10})

sorted_most_common_companies = sorted(patent_counter, key=patent_counter.get, reverse=True)

n_most_common_companies = sorted_most_common_companies
n_most_common_companies_df = data[data['Anmelder/Inhaber'].isin(sorted_most_common_companies)]

# Datennormalisierung
Die Spalter der Erfinder muss normalisiert werden, da in dieser mehrere Erfinder auftreten können. Diese werden am Semikolon getrennt und in einem weiteren Dataframe gespeichert.

In [4]:

normalized_data = []
# fill patents inventors if they are not filled
n_most_common_companies_df['Erfinder'] = n_most_common_companies_df['Erfinder'].fillna(value='no Inventor')
# loop over most common companies
for row in n_most_common_companies_df.itertuples():
    row_df = pd.DataFrame(row).transpose().drop(0, axis=1)
    row_df.columns = n_most_common_companies_df.columns
    
    # get inventors normalized 
    inventors = row_df["Erfinder"][0].split(';')       
    # create new line for every Inventor and applicant
    for inventor in inventors:
        normalized_row = [row_df["Veroeffentlichungs-Nummer"][0],inventor.strip(), row_df["Anmelder/Inhaber"][0]]
        normalized_data.append(normalized_row)

# drop duplicates based on inventor and applicant
normalized_data_df = pd.DataFrame(normalized_data, columns=["Veroeffentlichungs-Nummer", "Erfinder", "Anmelder/Inhaber"]) #.drop_duplicates(subset=["Erfinder", "Anmelder/Inhaber"])

normalized_data_df.to_csv("Datensatz_Erfinder_normalisiert.csv", sep=";")

<ipython-input-4-a37d15e478b2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_most_common_companies_df['Erfinder'] = n_most_common_companies_df['Erfinder'].fillna(value='no Inventor')


# Vorbereitung zu Visualisierung
Die Knoten werden mithilfe der Spalte Erfinder und Anmelder/Inhaber erstellt.

In [5]:
# create graph from edgelist
inventor_graph = nx.from_pandas_edgelist(normalized_data_df,
source = "Erfinder",
target = "Anmelder/Inhaber")

Die Knoten erhalten über Knotenattribute weitere Informationen, die später über Tooltips angezeigt werden können. Hierzu zählen die Namen der Erfinder und Unternehmen, die Zentralität sowie die Anzahl der Patente, an denen ein Erfinder mitgewirkt hat.

In [6]:
# add inventor as node attribute
inventor_dict = {tuple[2]: tuple[2] for tuple in normalized_data_df.itertuples()}
nx.set_node_attributes(inventor_graph, inventor_dict, "inventor")

# add company as node attribute
company_dict = {tuple[3]: tuple[3] for tuple in normalized_data_df.itertuples()}
nx.set_node_attributes(inventor_graph, company_dict, "company")

degree = nx.degree_centrality(inventor_graph)
degree_dict = {key:value*100 + 1 for key,value in degree.items()}
nx.set_node_attributes(inventor_graph, degree_dict, "centrality")

# add amount of patents to inventor / Company nodes will have 'null' values, because companies are not considered
patent_amount_counter = {}
patent_inventors = pd.DataFrame(normalized_data, columns=["Veroeffentlichungs-Nummer", "Erfinder", "Anmelder/Inhaber"]).drop_duplicates(subset=["Erfinder", "Veroeffentlichungs-Nummer"])

for tuple in patent_inventors.itertuples():    
    patent_amount_counter[tuple[2]] = patent_amount_counter.get(tuple[2], 0) + 1    
nx.set_node_attributes(inventor_graph, patent_amount_counter, "inventions")

Zusätzlich zu den vorherigen Informationen werden den Knoten Typen zugeordnet. Bei diesen Typen handelt es sich entweder Erfinder oder die Unternehmen. Diese Typen erhalten jeweils eine Knotenfarbe.

In [7]:
node_types = []
for node in inventor_graph.nodes:
    if node in normalized_data_df['Erfinder'].values:
        node_types.append('Erfinder')
    else:
        node_types.append('Unternehmen')

for i,n in enumerate(inventor_graph.nodes()):
    inventor_graph.nodes[n]['type'] = node_types[i]

# Visualisierung
Abschließend wird der Graph erstellt und die zuvor erläuterten Informationen dem Graphen übergeben. Nach Ausführung der nächsten Zelle kann über das Menü mit den drei Punkten neben dem Graphen und die dort angezeigte Option "Open in Vega Editor" der Graph mit interaktiven Optionen geöffnet werden.

In [8]:
alt.data_transformers.disable_max_rows()
chart = nxa.draw_networkx(
    inventor_graph,
    node_color= 'type',
    #cmap='viridis',
    edge_color='black',
    node_size= 25,
    node_tooltip = ['inventor', 'inventions', 'company']
).properties(
    width=800,
    height=800
).interactive()
chart

D:\Programme\Anaconda\envs\mining\lib\site-packages\altair\utils\core.py:185: UserWarning: I don't know how to infer vegalite type from 'integer-na'.  Defaulting to nominal.
  warnings.warn(


alt.LayerChart(...)